<a href="https://colab.research.google.com/github/Unknownboykim/Unknownboykim/blob/main/Internship_for_arkansas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from os import makedirs
from os import listdir
from shutil import copyfile
from random import seed
from random import random
import numpy as np
import pandas as pd

# visuals
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.image import imread
from PIL import Image

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,ConfusionMatrixDisplay

# Tensorflow
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense,MaxPooling2D,Dropout,Flatten,BatchNormalization,Conv2D
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install kaggle

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


Next Step: Unzip train.zip into the correct folder

In [ ]:
!kaggle competitions download -c dogs-vs-cats
!unzip dogs-vs-cats.zip -d /content/dogs-vs-cats


 99% 805M/812M [00:07<00:00, 82.8MB/s]
100% 812M/812M [00:07<00:00, 121MB/s] 
Archive:  dogs-vs-cats.zip
  inflating: /content/dogs-vs-cats/sampleSubmission.csv  
  inflating: /content/dogs-vs-cats/test1.zip  
  inflating: /content/dogs-vs-cats/train.zip  


In [ ]:
!mkdir -p /content/data/train
!unzip -q /content/dogs-vs-cats/train.zip -d /content/data/train

# Move images up one level

In [ ]:
!mv /content/data/train/train/* /content/data/train/

# Remove the now-empty nested folder

In [ ]:
!rmdir /content/data/train/train

Then: Run the Data Split Script

In [ ]:
import os, shutil, random

original_dir = '/content/data/train'
base_dir = '/content/data_split'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')

os.makedirs(train_dir + '/cat', exist_ok=True)
os.makedirs(train_dir + '/dog', exist_ok=True)
os.makedirs(val_dir + '/cat', exist_ok=True)
os.makedirs(val_dir + '/dog', exist_ok=True)

filenames = os.listdir(original_dir)
random.shuffle(filenames)

split_idx = int(0.8 * len(filenames))
for i, fname in enumerate(filenames):
    src = os.path.join(original_dir, fname)
    label = 'cat' if 'cat' in fname else 'dog'
    dst_dir = train_dir if i < split_idx else val_dir
    shutil.copy(src, os.path.join(dst_dir, label, fname))


PART 2: Build, Train, and Evaluate the CNN
Step 1: Import Required Libraries

In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Step 2: Define Your CNN Model

In [ ]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification (dog vs cat)
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Step 3: Load Your Training and Validation Data

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    '/content/data_split/train',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

val_data = val_datagen.flow_from_directory(
    '/content/data_split/val',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


Step 4: Train the Model

In [ ]:
history = model.fit(
    train_data,
    epochs=10,
    validation_data=val_data
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 1231s 2s/step - accuracy: 0.5920 - loss: 0.8467 - val_accuracy: 0.6574 - val_loss: 0.6229
Epoch 2/10
 70/625 ━━━━━━━━━━━━━━━━━━━━ 17:16 2s/step - accuracy: 0.6896 - loss: 0.5925

Step 5: Plot Accuracy and Loss

In [ ]:
# Accuracy plot
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title('Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

# Loss plot
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()


Step 6: Save the Trained Mode

In [ ]:
# Local
model.save('/content/cats_vs_dogs_model.h5')

# Or to Google Drive
# model.save('/content/drive/MyDrive/cats_vs_dogs_model.h5')


Step 7: Run a Prediction on a Single Image

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Load and preprocess image
img_path = '/content/data_split/val/cat/cat.4001.jpg'  # You can choose any image
img = image.load_img(img_path, target_size=(150, 150))
x = image.img_to_array(img) / 255.0
x = np.expand_dims(x, axis=0)

# Predict
prediction = model.predict(x)
print("Prediction:", "Dog 🐶" if prediction[0][0] > 0.5 else "Cat 🐱")
